This notebook test a roughdraft of the global structure of reading main (current county processing) along with adjacent county data to accomdate for edge-matching before triangulation. This notebook should serve as the basic structure before attempting to parallelize using dask.

In [1]:
from new_methods import *

In [2]:
""" Establish Map that will be used to identify adjacent FIPSSTCO """
# Read in csv
co = pd.read_csv('co_neighbor.csv', dtype={'src_FIPSSTCO': str, 'nbr_FIPSSTCO': str})
co = co[['src_FIPSSTCO', 'nbr_FIPSSTCO']]

# Groupby src and collect neighbors into list
co_gb = co.groupby('src_FIPSSTCO')['nbr_FIPSSTCO'].apply(list)
co_gb.head()

src_FIPSSTCO
01001                  [01021, 01047, 01051, 01085, 01101]
01003           [01025, 01053, 01097, 01099, 01129, 12033]
01005    [01011, 01045, 01067, 01109, 01113, 13061, 132...
01007           [01021, 01065, 01073, 01105, 01117, 01125]
01009           [01043, 01055, 01073, 01095, 01115, 01127]
Name: nbr_FIPSSTCO, dtype: object

In [3]:
co_gb['08013']

['08014', '08047', '08049', '08059', '08069', '08123']

In [7]:
""" Global Structure. The final script will not loop from a list of Main Counties but simply loop directly from
the main floodzone folder. The list is to test only cofips (48113 & 48439) as the main counties being processed in a 2-iteration loop.
Therefore, before finalizing: Make sure to reorient code under the FloodZone glob loop """

# Main Counties that will be processed
mco_ls = ['08013']

# Global identification of main co
for mco in mco_ls:
    print(mco)
    for mfi in glob.glob(f'{mco}_FZ.shp'):
        fips = os.path.basename(mfi).split('_')[0] # Retrieve current name of county being processed
        fz = gpd.read_file(os.path.abspath(mfi))# read current county flood zone
    fz = fz[['geometry']] 
        
    # Identify adjacent counties to current 
    adj_ls = co_gb[fips]
    
    # Loop through adj fips and read and collect flood zones/BFEs into single df
    adj_fz = gpd.GeoDataFrame()
    adj_bfes = gpd.GeoDataFrame()
    for a in adj_ls:
        for afi in glob.glob(f'{a}_FZ.shp'):
            afz = gpd.read_file(os.path.abspath(afi))
        adj_fz = pd.concat([adj_fz, afz], ignore_index=True)

        for ab in glob.glob(f'{a}_BFE.shp'):
            abfe = gpd.read_file(os.path.abspath(ab))
            abfe = abfe[['ELEV', 'geometry']]
        adj_bfes = pd.concat([adj_bfes, abfe])
        
            
    adj_fz = adj_fz[['geometry']]
    
    # Read Main County BFE
    for b in glob.glob(f'{mco}_BFE.shp'):
        mbfe = gpd.read_file(os.path.abspath(b))
        mbfe = mbfe[['ELEV', 'geometry']]
        
    # Add Main BFE to Adj_BFEs = All_BFE df (used in Edge-Matching process)
    all_bfe = pd.concat([adj_bfes, mbfe], ignore_index=True)

    # Determine correct utm projection for data
    crs = utm_code(fz)
    
    # Reproject data
    fz = fz.to_crs(crs) # Current County Flood Zone
    adj_fz = adj_fz.to_crs(crs) # All Adjacent Flood Zones
    mbfe = mbfe.to_crs(crs) # Current County BFE
    all_bfe = all_bfe.to_crs(crs) # All BFEs
    adj_bfes = adj_bfes.to_crs(crs) # Only Adj. BFEs
    
    """ Edge Identification. After Edges are identified they are seperated from the rest of the Counties Flood polygons. """
    # buffer fz geometry
    fz_buff = g(fz.buffer(3), crs)

    # spatial join with adjacent fz df (inner)
    fz_bsjoin = fz_buff.sjoin(adj_fz)

    # obtain ids that match and send to a list
    edge_ids = fz_bsjoin[['geometry']].drop_duplicates().index.to_list()

    # Locate ids in non-buffered df
    m_edges = fz.loc[fz.index.isin(edge_ids)]
    m_edges.reset_index(inplace=True, drop=True)

    # Seperate the Main Flood Zone polygons from edges
    m_fz = fz.loc[~fz.index.isin(edge_ids)]
    m_fz.reset_index(inplace=True, drop=True)

    """ Edge Matching and Triangulation Process """
    edge_triangles = gpd.GeoDataFrame()
    miss = gpd.GeoDataFrame()
    # Interpolate Points -- Convert to seperate gdf
    points = 12
    m_edges['line_geom'] = m_edges.apply(lambda x: x.geometry.boundary, axis=1)
    for i, e in m_edges.iterrows():
        try:
            print(i)
            # Create points along edge and buffer
            pts = [e.line_geom.interpolate(i/points, normalized=True) for i in range(1, points)]
            pts = g(pts, crs)
            pts_buff = g(pts.buffer(3).geometry, crs)
            
            # Spatial join df to Adj. Edge Poly df
            pjoin = pts_buff.sjoin(adj_fz)

            # Identify which index occurs only once // Most often?
            adjId = pjoin['index_right'].value_counts().idxmax()

            # Locate adjId in Tar
            adj_edge = adj_fz.loc[adjId]

            # Clean up d df by removing line_geom
            e = e[['geometry']]

            # Merge with Adj. COunty Edge df to get Adj. Geometry + convert to gdf
            e_join = pd.concat([e, adj_edge], ignore_index=True)
            e_join = g(e_join, crs)
            
            # Union both geometries and pass to seperate gdf + annoying (but necessary) buffer
            edge = g(unary_union(e_join.geometry.to_list()), crs)
            edge_buffer = g(edge.buffer(1), crs) 

            # Locate corresponding BFEs to Union Edge Poly
            bfe_set = all_bfe.sjoin(edge_buffer)
            bfe_set = bfe_set.drop_duplicates()
            bfe_set = bfe_set[['ELEV', 'geometry']]
            bfe_set = remove_multiline_BFE(bfe_set)

            # Getting Z-geom for BFE Points
            bfe_pts_utm, bfe_pts_84 = bfe_zpts(bfe_set, crs=crs)

            # FSP Simplify, Interpolation, and Z-geom
            fsp_i_pts = fsp_pts(edge, bfe_pts=bfe_pts_utm, bfe_set=bfe_set, diff_area=1000, crs=crs)

            if fsp_i_pts is not None:
                print('Creating more points..')
                # Elevation interpolation
                fsp_i_pts = IDW(fsp_i_pts, bfe=bfe_set, power=2)
                
                if len(fsp_i_pts) > 0:
                    # Insert ELEV field in geometry
                    fsp_i_pts = ELEV_2geom(fsp_i_pts)
                    
                # Concat and Triangulate
                all_pts = pd.concat([bfe_pts_84, fsp_i_pts], ignore_index=True)
                all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

                tin = triangulate(all_pts_multigeom)
                tin_df = g(tin, 4326)

                # Extract Geom
                final_tin = extract_geom(tin_df)

                # Tag TIN df with corresponding Flood Zone
                #final_tin['FZ'] = i 
                
                edge_triangles = pd.concat([edge_triangles, final_tin], ignore_index=True)

            else:
                print('Using only BFE pts')
                # Concat and Triangulate
                all_pts_multigeom = MultiPoint(bfe_pts_84.geometry.to_list())

                tin = triangulate(all_pts_multigeom)
                tin_df = g(tin, 4326)

                # Extract Geom
                final_tin = extract_geom(tin_df)  

                # Tag TIN df with corresponding Flood Zone
                #final_tin['FZ'] = i    

                # Collect     
                edge_triangles = pd.concat([edge_triangles, final_tin], ignore_index=True)

        except:
            print('Problem with Poly: ', i)
            p_er_dict = {i: e.geometry}
            error = gpd.GeoSeries(p_er_dict, crs=crs)
            miss = pd.concat([miss, error])
            continue
    if len(miss) > 0:
        # Reconfgure 'miss' series to geodataframe
        miss.rename(columns={0: 'geometry'}, inplace=True)
        miss = miss[['geometry']]
        miss = miss.set_geometry('geometry', crs=crs)

        # Add miss df to Main FZ df
        m_fz = pd.concat([m_fz, miss], ignore_index=True)

    """ Clip edge Triangulation to Current County Edges for final edge dataframe """
    if len(edge_triangles) > 0:
        fz84 = fz.to_crs(4326)
        edge_tri = edge_triangles.overlay(fz84) # Eventually merge this df to main triangulation for final output

    """ Main AE Trianglualtion Phase """
    # Iterate through each FSP polygon
    main_triangles = gpd.GeoDataFrame()
    poly_errors = gpd.GeoSeries()
    for i, f in m_fz.iterrows():
        try:
            f = g(f.geometry, crs)
            bfe_set = mbfe.sjoin(f)
            
            bfe_set = bfe_set[['ELEV', 'geometry']]
            bfe_set = remove_multiline_BFE(bfe_set)
            
            print('POLY INDEX: ', i)
            #Ignoring Potential Polygon Slivers due to extension of BFEs overlapping small poritions of FSP
            if len(f.sjoin(bfe_set)) == 0:
                continue
            # Potential Island poly which will not intersect BFEs. Ignore!
            elif bfe_set.shape[0] == 0:
                continue
            else:
                
                # Getting Z-geom for BFE Points
                bfe_pts_utm, bfe_pts_84 = bfe_zpts(bfe_set, crs=crs)

                # FSP Simplify, Interpolation, and Z-geom
                fsp_i_pts = fsp_pts(f, bfe_pts=bfe_pts_utm, bfe_set=bfe_set, diff_area=1000, crs=crs)
                
                if fsp_i_pts is not None:
                    print('Creating more points..')
                    # Elevation interpolation
                    fsp_i_pts = IDW(fsp_i_pts, bfe=bfe_set, power=2)
                    
                    if len(fsp_i_pts) > 0:
                        # Insert ELEV field in geometry
                        fsp_i_pts = ELEV_2geom(fsp_i_pts)

                    # Concat and Triangulate
                    all_pts = pd.concat([bfe_pts_84, fsp_i_pts], ignore_index=True)
                    all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

                    tin = triangulate(all_pts_multigeom)
                    tin_df = g(tin, 4326)
                
                    # Extract Geom
                    final_tin = extract_geom(tin_df)
                    
                    #final_tin = final_tin.overlay(f) #(final_tin.geometry.centroid.within(f.geometry[0])) |
                                
                    main_triangles = pd.concat([main_triangles, final_tin], ignore_index=True)

                else:
                    print('Using only BFE pts')
                    # Concat and Triangulate
                    all_pts_multigeom = MultiPoint(bfe_pts_84.geometry.to_list())

                    tin = triangulate(all_pts_multigeom)
                    tin_df = g(tin, 4326)
                
                    # Extract Geom
                    final_tin = extract_geom(tin_df)
                    #final_tin = final_tin.overlay(f) #(final_tin.geometry.centroid.within(f.geometry[0])) |
                                        
                    main_triangles = pd.concat([main_triangles, final_tin], ignore_index=True)

            
        except:
            print('Problem with Poly: ', i)
            p_er_dict = {i: f['geometry'][0]}
            error = gpd.GeoSeries(p_er_dict, crs=crs)
            poly_errors = pd.concat([poly_errors, error])
            continue

    if len(poly_errors) > 0:
        poly_errors = poly_errors.to_crs(4326)
        poly_errors.to_file(f'{fips}_polygon_errors.shp')

    # Clean up Main Flood Zone Triangles and Merge with edges for Final Output
    all_tri = pd.concat([main_triangles, edge_tri], ignore_index=True)
    all_tri = all_tri[['geometry', 'pt_0_LAT', 'pt_0_LONG', 'pt_0_Z', 'pt_1_LAT', 'pt_1_LONG',
       'pt_1_Z', 'pt_2_LAT', 'pt_2_LONG', 'pt_2_Z']]
    # Write outputs
    all_tri.to_file(f'{fips}_Triangles.shp')

   


08013
0
Problem with Poly:  0
1
Creating more points..
2
Creating more points..
3
Creating more points..
4
Creating more points..
5
Creating more points..
6
Using only BFE pts
7
Using only BFE pts
8
Using only BFE pts
9
Creating more points..
10
Using only BFE pts
11
Creating more points..
12
Problem with Poly:  12
13
Creating more points..
14
Using only BFE pts
15
Using only BFE pts
16
Creating more points..
17
Creating more points..
18
Creating more points..
19
Creating more points..
20
Creating more points..
21
Creating more points..


KeyboardInterrupt: 

fz.to_file('fz.shp')
adj_fz.to_file('adj_fz.shp')
mbfe.to_file('mbfe.shp')
all_bfe.to_file('all_bfe.shp')
adj_bfes.to_file('adj_bfe.shp')


m_edges.to_file('m_edges.shp')
m_fz.to_file('m_fz.shp')

In [12]:
edge_triangles

""


In [9]:
all_tri.to_file('all_tri.shp')

Index(['geometry', 'pt_0_LAT', 'pt_0_LONG', 'pt_0_Z', 'pt_1_LAT', 'pt_1_LONG',
       'pt_1_Z', 'pt_2_LAT', 'pt_2_LONG', 'pt_2_Z'],
      dtype='object')